In [1]:
import pandas as pd
import json
import openai
import os
from openai import OpenAI
import time
from dotenv import load_dotenv

In [2]:
file_path = "/Users/priyal/Documents/honours/datasets/qa_dataset_65yo-500.csv"  
df = pd.read_csv(file_path)

jsonl_path = "/Users/priyal/Documents/honours/datasets/qa_dataset_65yo-500.jsonl"

with open(jsonl_path, "w") as f:
    for _, row in df.iterrows():
        jsonl_entry = {
            "messages": [
                {"role": "system", "content": "You are an expert in Ayurveda."},
                {"role": "user", "content": f"{row['instruction']}\n{row['input']}"},
                {"role": "assistant", "content": row["output"]},
            ]
        }
        f.write(json.dumps(jsonl_entry) + "\n")

print(f"Dataset converted and saved as {jsonl_path}")

Dataset converted and saved as /Users/priyal/Documents/honours/datasets/qa_dataset_65yo-500.jsonl


In [3]:
load_dotenv("/Users/priyal/Documents/honours/.env/key.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
dataset_path = "/Users/priyal/Documents/honours/json/qa_dataset_65yo-500.jsonl"

In [5]:
with open(dataset_path, "rb") as file:
    response = client.files.create(file=file, purpose="fine-tune")

file_id = response.id
print(f"File ID: {file_id}")

File ID: file-KKPwZwzk2yeGxbJVWTNmkJ


In [6]:
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={"batch_size": 4, "n_epochs": 5},
)

fine_tune_id = fine_tune_response.id
print(f"Fine-tune ID: {fine_tune_id}")


Fine-tune ID: ftjob-GQLEgwIrAjsaUClQ7PaHhSrM


In [7]:
while True:
    response = client.fine_tuning.jobs.retrieve(fine_tune_id)
    status = response.status

    print(f"Status: {status}")

    if status in ["succeeded", "failed", "cancelled"]:
        break

    time.sleep(60)

print(f"Fine-tuning job completed with status: {status}")

Status: validating_files
Status: validating_files
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: succeeded
Fine-tuning job completed with status: succeeded
